In [1]:
#JSONとOAuth認証用のライブラリを使用
import json
from requests_oauthlib import OAuth1Session
# import setting
import tweepy
import os
import pandas as pd
# twitter = OAuth1Session(setting.CONSUMER_KEY, setting.CONSUMER_SECRET, setting.ACCESS_TOKEN, setting.ACCESS_SECRET)

In [2]:
df_f = pd.read_json('../tweet_extructor/twitterJSA_data.json')


In [3]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_f, test_size=0.2, random_state=42)

In [4]:
# train_df.to_csv("./train_twitterJSA_data.csv")
# test_df.to_csv("./test_twitterJSA_data.csv")

In [5]:
#
import torch
from torch import nn
from transformers import BertTokenizer, BertModel, BertForMaskedLM, tokenization_bert
import numpy as np
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
import random

model = BertModel.from_pretrained('bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers')
tokenizer_bert  = BertTokenizer("bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt",
                               do_lower_case=False, do_basic_tokenize=False)

In [6]:
import re
import string
import torchtext
# from bert import BertTokenizer
# from bert import BertTokenizer

def preprocessing_text(text):
    text = re.sub("<br/>", "", text)
    
    for p in string.punctuation:
        if (p==".")or (p==",") or (p=="[UNK]"):
            continue
        else:
            text=text.replace(p, "")

    return text

def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)
    return ret

max_length = 256



In [18]:
# TEXT = data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, lower=True, init_token="[cls]", eos_token="[sep]", pad_token='[pad]', unk_token='[unk]')


TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=False, include_lengths=True, batch_first=True, fix_length=max_length, init_token="[CLS]", 
                            eos_token="[SEP]", pad_token='[PAD]', unk_token='[UNK]')

LABEL = data.Field(sequential=False, use_vocab=False)

In [19]:
!head -c 2500 ../tweet_extructor/twitterJSA_data.json # 文字化けは気にしなくて良いはず

[{"id": 10025, "topic": 10000, "status": 522407718091366400, "label": [0, 0, 1, 1, 0], "text": "\u30a8\u30af\u30b9\u30da\u30ea\u30a2\u306eGPS\u5357\u5317\u304c\u9006\u306b\u306a\u308b\u306e\u306f\u30c7\u30d5\u30a9\u3060\u3063\u305f\u306e\u304b\u3002"}, {"id": 10026, "topic": 10000, "status": 522407768003592192, "label": [0, 0, 1, 0, 0], "text": "xperia\u3067\u30b9\u30af\u30d5\u30a7\u30b9\u7cde\n\u53cd\u5fdc\u9045\u3044\u3093\u3060\u3088\u7cde\u304c"}, {"id": 10027, "topic": 10000, "status": 522408018642628609, "label": [0, 0, 1, 1, 0], "text": "\u590f\u6625\u90fd\u304c\u6301\u3063\u3066\u305f\u30a8\u30af\u30b9\u30da\u30ea\u30a2\u3082\u4eca\u4f7f\u3046\u306b\u306f\u8f9b\u3044"}, {"id": 10028, "topic": 10000, "status": 522408394871672832, "label": [0, 0, 0, 1, 0], "text": ""}, {"id": 10029, "topic": 10000, "status": 522408454778929153, "label": [0, 0, 0, 1, 0], "text": ""}, {"id": 10030, "topic": 10000, "status": 522408539814260736, "label": [0, 1, 0, 0, 0], "text": ""}, {"id": 10031, "t

In [53]:
# 今回は2つのjsonデータを読み込む

train_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./',
    train="train_twitterJSA_data.csv",
    test="test_twitterJSA_data.csv",
    format='csv', fields= [("text", TEXT), ("label", LABEL)] )



In [54]:
# train_ds, val_ds = dataset[0].split() # ので、この式を実行するとエラー仕方ないので、下を実行
# train_ds= dataset[0].split()

In [55]:
!ls bert

Japanese_L-12_H-768_A-12_E-30_BPE_transformers


In [56]:
len(tokenization_bert.load_vocab(vocab_file="./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt"))

32006

In [57]:
# BERT用の単語辞書を辞書型変数に用意します
# from bert import  load_vocab
# vocab_bert, ids_to_tokens_bert = tokenization_bert.load_vocab(vocab_file="./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt")
vocab_bert = tokenization_bert.load_vocab(vocab_file="./bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt")

# 1度適当にbuild_vocabでボキャブラリーを作成してから、BERTのボキャブラリーを上書きします
TEXT.build_vocab("", min_freq=1)
TEXT.vocab.stoi = vocab_bert

In [58]:
train_dl = torchtext.data.Iterator(
#     train_ds[0], batch_size=3, train=True)
    train_ds, batch_size=3, train=True)

# ここで結局エラー・・・

In [59]:
print(len(train_ds))

428017


In [60]:
TEXT.vocab.stoi

OrderedDict([('[PAD]', 0),
             ('[UNK]', 1),
             ('[CLS]', 2),
             ('[SEP]', 3),
             ('[MASK]', 4),
             ('の', 5),
             ('、', 6),
             ('。', 7),
             ('に', 8),
             ('は', 9),
             ('を', 10),
             ('が', 11),
             ('と', 12),
             ('で', 13),
             ('年', 14),
             ('・', 15),
             ('（', 16),
             ('）', 17),
             ('さ', 18),
             ('して', 19),
             ('した', 20),
             ('いる', 21),
             ('する', 22),
             ('も', 23),
             ('「', 24),
             ('」', 25),
             ('月', 26),
             ('から', 27),
             ('れた', 28),
             ('日', 29),
             ('こと', 30),
             ('し', 31),
             ('である', 32),
             ('れて', 33),
             ('や', 34),
             ('２', 35),
             ('１', 36),
             ('いた', 37),
             ('ある', 38),
             ('『', 39),
             ('』'

In [61]:
# 動作確認 検証データのデータセットで確認
batch = next(iter(train_dl))
print(batch.text)
print(batch.label)

(tensor([[2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [62]:
# for i in train_dl:
#     print(i)

In [63]:
class BertForTwitterSuzuki(nn.Module):
    '''BERTモデルにTwitterのポジ・ネガ系５分類を判定する部分をつなげたモデル'''

    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()

        # BERTモジュール
        self.bert = net_bert  # BERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元、出力はポジ・ネガの2つ
        self.cls = nn.Linear(in_features=768, out_features=5)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        token_type_ids： [batch_size, sequence_length]の、各単語が1文目なのか、2文目なのかを示すid
        attention_mask：Transformerのマスクと同じ働きのマスキングです
        output_all_encoded_layers：最終出力に12段のTransformerの全部をリストで返すか、最後だけかを指定
        attention_show_flg：Self-Attentionの重みを返すかのフラグ
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        if attention_show_flg == True:
            '''attention_showのときは、attention_probsもリターンする'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
        out = self.cls(vec_0)

        # attention_showのときは、attention_probs（1番最後の）もリターンする
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out

In [72]:
# ミニバッチの1文目を確認してみる
text_minibatch_1 = (batch.text[0][1]).numpy()

# IDを単語に戻す
text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)

print(text)

['[CLS]', '[UNK]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 